<a href="https://colab.research.google.com/github/anselmo-pitombeira/Notebooks/blob/master/Problema_do_transporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Instalação do pyomo e do solver para o colab ##
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc

     |████████████████████████████████| 9.7 MB 4.1 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155676 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unp

In [2]:
## Modelo matemático ##
## Min 162x11 + 247x12 + 117x21 + 193x22 + 131x31 + 185x32 ##
## S.a x11 + x12 = 1000 ##
##     x21 + x22 = 1500 ##
##     x31 + x32 = 1200 ##
##     x11 + x21 + x31 = 2300 ##
##     x12 + x22 + x32 = 1400 ##

## Caso a implementação esteja sendo feita por uma IDE: ##
## Abra o Anaconda Prompt e insira as seguintes chamadas: ##
## conda install -c conda-forge pyomo ##
## conda install -c conda-forge glpk ##


## Importando a biblioteca pyomo ##
import pyomo.environ as pyEnv

## Dados do problema ##
custos_ij = [[162, 247], [117, 193], [131, 185]]
suprimentos = [1000, 1500, 1200]
demandas = [2300, 1400]

m = len(suprimentos)
n = len(demandas)

## Declarando o modelo através da biblioteca ##
modelo = pyEnv.ConcreteModel()

## Criando Indices ##
modelo.Indices_fabricas = pyEnv.RangeSet(m)
modelo.Indices_clientes = pyEnv.RangeSet(n)

## Criando as variáveis ##
modelo.x = pyEnv.Var(modelo.Indices_fabricas, modelo.Indices_clientes, within = pyEnv.NonNegativeReals)

modelo.Custo_transporte = pyEnv.Param(modelo.Indices_fabricas, modelo.Indices_clientes, initialize = lambda modelo, i, j: custos_ij[i-1][j-1])
modelo.Suprimentos = pyEnv.Param(modelo.Indices_fabricas, initialize = lambda modelo, i: suprimentos[i-1])
modelo.Demandas = pyEnv.Param(modelo.Indices_clientes, initialize = lambda modelo, j: demandas[j-1])

## Criando a Função Objetivo ##
modelo.Objetivo = pyEnv.Objective(expr = sum(modelo.x[i,j] * modelo.Custo_transporte[i,j] for i in modelo.Indices_fabricas for j in modelo.Indices_clientes), sense = pyEnv.minimize)

## Criando as restrições ##
def rest1(modelo, i):
  return sum(modelo.x[i,j] for j in modelo.Indices_clientes) == modelo.Suprimentos[i]

def rest2(modelo, j):
  return sum(modelo.x[i,j] for i in modelo.Indices_fabricas) == modelo.Demandas[j]

modelo.rest1 = pyEnv.Constraint(modelo.Indices_fabricas, rule = rest1)
modelo.rest2 = pyEnv.Constraint(modelo.Indices_clientes, rule = rest2)


In [3]:
## Chamando o Solver Coin-OR CBC##
solver = pyEnv.SolverFactory('cbc')
result_objetivo = solver.solve(modelo, tee = True)   ##tee = True manda imprimir o log do solver

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Aug 21 2017 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpfy6o24zc.pyomo.lp -stat=1 -solve -solu /tmp/tmpfy6o24zc.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 2 (-4) rows, 3 (-4) columns and 6 (-7) elements
Statistics for presolved model


Problem has 2 rows, 3 columns (3 with objective) and 6 elements
Column breakdown:
0 of type 0.0->inf, 3 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
2 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
0 of type L other, 0 of type Range 0.0->1.0, 0 of type Range other, 
0 of type Free 
Presolve 2 (-4) rows, 3 (-4) columns and 6 (-7) elements
0  Obj 750907.6 Primal inf 4400.2001 (2) Dual inf 215 (3)

In [4]:
## Printando o resultado ##
print("Quantidades a transportar:")
lista = list(modelo.x.keys())

print()
for i in lista:
  if modelo.x[i]() != 0:
    print(i, '---', modelo.x[i]())
print()
print('Valor da função objetivo (custo mínimo) =', modelo.Objetivo())

Quantidades a transportar:

(1, 1) --- 1000.0
(2, 1) --- 1300.0
(2, 2) --- 200.0
(3, 2) --- 1200.0

Valor da função objetivo (custo mínimo) = 574700.0
